In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("Freq_Dataframes") \
        .master('local[*]') \
        .getOrCreate()

#spark = SparkSession.builder.appName("Dataframes") \
#        .getOrCreate()

In [ ]:
from pyspark.sql.functions import split, lower, explode, \
                                  regexp_replace, \
                                  size

In [ ]:
df = spark.createDataFrame([('¡Esta es una linea!',), ('¿esta es otra linea?',)], ['col1'])
df.show()

In [ ]:
df_limpia = df.withColumn(
    'col1', 
    regexp_replace('col1', '[^a-zA-Z0-9 ]', '')
)
df_limpia.show()

In [ ]:
df_words = df_limpia.select( 
                        explode( 
                            split( 
                                lower(df_limpia.col1), 
                                " "
                            ) 
                        ).alias("palabras") 
)

df_words.show()

In [ ]:
freq = df_words.groupBy('palabras').count()

freq.show()

# Caso de dos documentos

In [ ]:
data = [
    ('1', 'Este es el libro sobre Frankestein.\n'
          '¡Este libro tiene dos lineas!'),
    ('2', 'Este es el libro sobre la Bella y la Bestia.\n'
          ' Este libro tiene tres lineas,\n'
          '¡Chulada de libro!  ¡chulada de Bestia!')
]

schema = 'doc:string, texto:string'

In [ ]:
libros = spark.createDataFrame(data, schema)
libros.show(truncate=False)

In [ ]:
def clean(df, col:str):
    df_limpia = df.withColumn(
        col, 
        regexp_replace(col, '[^a-zA-Z0-9 \n]', '')
    )
    return df_limpia

In [ ]:
def aPalabras(df, col:str):
    df_words = df.select( 
        explode( 
            split( 
                lower(df[col]), 
                "\\s+"
            ) 
        ).alias("palabras") 
    )
    return df_words

In [ ]:
def aPalabras2(df, col:str):
    df_words = df.withColumn( 
        col,
        explode( 
            split( 
                lower(df[col]), 
                "\\s+"
            ) 
        ) 
    )
    return df_words

## Limpiar el libro de caracteres que no pertenescan a palabras

In [ ]:
libros_clean = clean(libros, 'texto')
libros_clean.show(truncate=False)

## Separar en palabras

In [ ]:
new_df = aPalabras( libros_clean, 'texto')
new_df.show(40)

## Elproblema con la función anterior era q afectaba la segunda columna pero no consideraba la primer columna donde se incluye el libro

In [ ]:
df_palabras = aPalabras2( libros_clean, 'texto')
df_palabras.show(30)

## Tamaño de los libros

In [ ]:
df_size = df_palabras.groupBy('doc').count()
df_size.show()

## Agrupacion

In [ ]:
frecuencias = df_palabras.groupBy('doc', 'texto').count()
frecuencias.show(30)

## Houston! Necesitamos una normalizacion
### Tomar en cuenta el tamaño de los libros desde el principio

In [ ]:
def get_words(df, col:str):
    df_words = df.withColumn( 
        col,
        explode( 
            split( 
                lower(df[col]), 
                "\\s+"
            ) 
        ) 
    )
    return df_words

In [ ]:
def get_words_list(df, col:str):
    df_words = df.withColumn( 
        col, 
        split( 
            lower(df[col]), 
            "\\s+"
        )
    )
    return df_words

In [ ]:
def get_words_size(df, col:str):
    df_words = df.withColumn( 
        col, 
        split( 
            lower(df[col]), 
            "\\s+"
        )
    ).withColumn(
        'size',
        size(col)
    )
    # colocar palabras de la lista una bajo la otra
    df_words = df_words.withColumn(
        col,
        explode(col)
    )
    return df_words

In [ ]:
get_words_list( libros_clean, 'texto').show(30)

In [ ]:
df_words_size = get_words_size( libros_clean, 'texto')
df_words_size.show(30)

In [ ]:
df_words_size = df_words_size.withColumn('size', 1/df_words_size.size)
df_words_size.show()

## Frecuencias normalizadas (TF)

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df_TF = df_words_size.groupBy('doc', 'texto').agg( F.sum('size').alias('TF') )
df_TF.show(30)

## --------------------------------------------------
## Segunda opcion
### Aquí les muestro como hacerlo deotra manera; utilizando funciones que actuan por renglon

In [ ]:
frecuencias.show()
frecuencias.printSchema()

df_size.show()

### Ejemplo de una función que actua por renglón

In [ ]:
from pyspark.sql.types import FloatType

In [ ]:
@F.udf(FloatType())
def multiplicar(doc, count):
    if doc == '1':
        return count * 3.0
    elif doc == '2':
        return count * 2.0

In [ ]:
frecuencias.withColumn('mult', multiplicar('doc', 'count')).show()

## Ahora si, a hacer lo nuestro

In [ ]:
my_dict = {row['doc']:row['count'] for row in df_size.collect()} 
my_dict

In [ ]:
@F.udf(FloatType())
def get_TM(doc, count):
    return count / float(my_dict[doc])

In [ ]:
frecuencias.withColumn('TM', get_TM('doc', 'count')).show()